### Import dependencies

### Import dependencies

In [ ]:
# %pip install spacy datasets span-marker scikit-learn

In [ ]:
# %pip install numpy 2.0.0

In [ ]:
# %pip install pandas 2.2.2

In [2]:
import os
from dotenv import load_dotenv

from openai import OpenAI

from langchain.agents import AgentExecutor
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import SupabaseVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain import hub
from langchain_core.tools import tool

from qdrant_client import QdrantClient
from qdrant_client.http.models import Filter, SparseVector
from fastembed import TextEmbedding, SparseTextEmbedding
from langchain.vectorstores import Qdrant

In [3]:
# attach to the same event-loop
import nest_asyncio

nest_asyncio.apply()

In [4]:
from langchain.evaluation import (
    EvaluatorType,
    load_evaluator,
    CriteriaEvalChain,
    LabeledCriteriaEvalChain
)
from langchain.schema import Document
import asyncio

In [5]:
import json
from pathlib import Path

In [6]:
import pandas as pd
import numpy as np

### Load environment variables

In [ ]:
# Load environment variables
load_dotenv()
QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
COLLECTION_NAME = os.getenv("QDRANT_COLLECTION_NAME")
OPENAI_API_KEY= os.getenv("OPENAI_API_KEY")

In [8]:
# Initialize dense and sparse embedding models
dense_model = TextEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
sparse_model = SparseTextEmbedding(model_name="Qdrant/bm42-all-minilm-l6-v2-attentions")

In [9]:
# Initialize Qdrant client
qdrant_client = QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY,
)


/tmp/ipykernel_20736/2748386960.py:2: UserWarning: Api key is used with an insecure connection.
  qdrant_client = QdrantClient(


In [10]:
# Initialize LLM
llm = ChatOpenAI(temperature=0)



In [11]:
# fetch the prompt from the prompt hub
prompt = hub.pull("hwchase17/openai-functions-agent")

/home/agorricho/anaconda3/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [12]:
from qdrant_client import QdrantClient, models
from langchain.agents import tool

@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Perform hybrid (dense + sparse) search using Qdrant Query API."""

    # Generate dense and sparse embeddings
    dense_emb = list(dense_model.embed([query]))[0]
    sparse_emb = list(sparse_model.embed([query]))[0]
    sparse_vector = models.SparseVector(
        indices=sparse_emb.indices.tolist(),
        values=sparse_emb.values.tolist()
    )

    
    # Fusion query
    context = qdrant_client.query_points(
        collection_name=COLLECTION_NAME,
        prefetch=[
            models.Prefetch(
                query=sparse_vector,
                using="sparse",
                limit=20,
            ),
            models.Prefetch(
                query=dense_emb,
                using="dense",
                limit=20,
            ),
        ],
        query=models.FusionQuery(fusion=models.Fusion.RRF),
        with_payload=True,
        # with_vector=False,
        limit=5
    )

    # Format result
    serialized = "\n\n".join(
        (f"ID: {pt.id}\
        \nScore: {pt.score}\
        \nText: {pt.payload['text']}\
        \nSource: {pt.payload['file_name']}\
        \nPage: {pt.payload['page_number']}")
        for pt in context.points
    )

    return serialized, context


In [13]:
query='what are the services offered'

In [14]:
serialized = retrieve(query)

/tmp/ipykernel_20736/1996469919.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  serialized = retrieve(query)


In [15]:
print(serialized)

ID: 12        
Score: 0.64285713        
Text: In a Full-Service relationship with a dedicated ﬁnancial professional, you have access to a full suite of TIS brokerage and TAS 
investment advisory products and services. Minimum asset balance requirements apply. You have a direct relationship with
your ﬁnancial professional for investment strategies, recommendations, and guidance speciﬁc to your needs and goals. Full- 
Service brokerage accounts are eligible to invest in equities, ﬁxed income, ETFs, mutual funds, annuities, structured products,
alternative investments, and other securities on our Platform which TIS makes available to such accounts. 
In a Full-Service relationship with the Client Advisory Center (“CAC”), you have access to a more limited menu of TIS brokerage
products and services from which a CAC ﬁnancial professional can make recommendations, including mutual funds, annuities,
brokered CDs, Treasuries, structured products, and certain advisory programs oﬀered through TA

In [19]:
sparse_embedding

SparseEmbedding(values=array([0.34146956, 0.26927151]), indices=array([1342301861, 1401524087]))

In [20]:
# Create sparse vector
sparse_vector = SparseVector(
    indices=sparse_embedding.indices.tolist(),
    values=sparse_embedding.values.tolist()
    )

In [21]:
sparse_vector

SparseVector(indices=[1342301861, 1401524087], values=[0.34146955540420515, 0.2692715119050256])

In [22]:
# Fusion query
response = qdrant_client.query_points(
    collection_name=COLLECTION_NAME,
    prefetch=[
        models.Prefetch(
            query=sparse_vector,
            using="sparse",
            limit=20,
        ),
        models.Prefetch(
            query=dense_emb,
            using="dense",
            limit=20,
        ),
    ],
    query=models.FusionQuery(fusion=models.Fusion.RRF),
    with_payload=True,
    # with_vector=False,
    limit=5
)

In [24]:
len(response.points)

5

In [25]:
response.points[4].score

0.33333334

In [26]:
# Format result
serialized_1 = "\n\n".join(
    (f"ID: {pt.id}\
     \nScore: {pt.score}\
     \nText: {pt.payload['text']}\
     \nSource: {pt.payload['file_name']\
     }")
    for pt in response.points
)

print(serialized_1)

ID: 12     
Score: 0.64285713     
Text: In a Full-Service relationship with a dedicated ﬁnancial professional, you have access to a full suite of TIS brokerage and TAS 
investment advisory products and services. Minimum asset balance requirements apply. You have a direct relationship with
your ﬁnancial professional for investment strategies, recommendations, and guidance speciﬁc to your needs and goals. Full- 
Service brokerage accounts are eligible to invest in equities, ﬁxed income, ETFs, mutual funds, annuities, structured products,
alternative investments, and other securities on our Platform which TIS makes available to such accounts. 
In a Full-Service relationship with the Client Advisory Center (“CAC”), you have access to a more limited menu of TIS brokerage
products and services from which a CAC ﬁnancial professional can make recommendations, including mutual funds, annuities,
brokered CDs, Treasuries, structured products, and certain advisory programs oﬀered through TAS. Ind

In [27]:
serialized_2 = "\n\n".join(
    (f"Source: {pt.payload['file_name']}\
     \n" f"Content: {pt.payload['text']}")
    for pt in response.points
    )

print(serialized_2)

Source: tis-investing-guide.pdf     
Content: In a Full-Service relationship with a dedicated ﬁnancial professional, you have access to a full suite of TIS brokerage and TAS 
investment advisory products and services. Minimum asset balance requirements apply. You have a direct relationship with
your ﬁnancial professional for investment strategies, recommendations, and guidance speciﬁc to your needs and goals. Full- 
Service brokerage accounts are eligible to invest in equities, ﬁxed income, ETFs, mutual funds, annuities, structured products,
alternative investments, and other securities on our Platform which TIS makes available to such accounts. 
In a Full-Service relationship with the Client Advisory Center (“CAC”), you have access to a more limited menu of TIS brokerage
products and services from which a CAC ﬁnancial professional can make recommendations, including mutual funds, annuities,
brokered CDs, Treasuries, structured products, and certain advisory programs oﬀered through TAS

In [28]:
tools = [retrieve]
agent = create_tool_calling_agent(llm, tools, prompt)

# create the agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [29]:
class ContextBasedEvaluator:
    def __init__(self, llm_model="gpt-3.5-turbo", temperature=0):
        if llm_model is None:
            self.llm = ChatOpenAI(temperature=temperature)
        elif isinstance(llm_model, str):
            self.llm = ChatOpenAI(model=llm_model, temperature=temperature)
        else:
            # Assume it's already an LLM instance
            self.llm = llm_model
        self._setup_evaluators()

    def _setup_evaluators(self):
        self.correctness = LabeledCriteriaEvalChain.from_llm(self.llm, criteria="correctness")
        self.relevance = CriteriaEvalChain.from_llm(self.llm, criteria="relevance")
        self.faithfulness = CriteriaEvalChain.from_llm(self.llm, criteria="coherence")
        self.helpfulness = CriteriaEvalChain.from_llm(self.llm, criteria="helpfulness")

    def evaluate(
        self,
        question: str,
        generated_answer: str,
        context: str,
        reference_answer: str = None
    ):
        results = {
            "question": question,
            "generated_answer": generated_answer,
            "reference_answer": reference_answer,
            "context": serialized_1
        }

        try:
            f = self.faithfulness.evaluate_strings(
                prediction=generated_answer,
                input=f"Context: {context}\nQuestion: {question}"
            )
            results.update({"faithfulness_score": f["score"], "faithfulness_reasoning": f["reasoning"]})
        except Exception as e:
            results.update({"faithfulness_score": None, "faithfulness_reasoning": str(e)})

        try:
            r = self.relevance.evaluate_strings(prediction=generated_answer, input=question)
            results.update({"relevance_score": r["score"], "relevance_reasoning": r["reasoning"]})
        except Exception as e:
            results.update({"relevance_score": None, "relevance_reasoning": str(e)})

        try:
            h = self.helpfulness.evaluate_strings(
                prediction=generated_answer,
                input=f"Context: {context}\nQuestion: {question}"
            )
            results.update({"helpfulness_score": h["score"], "helpfulness_reasoning": h["reasoning"]})
        except Exception as e:
            results.update({"helpfulness_score": None, "helpfulness_reasoning": str(e)})

        if reference_answer:
            try:
                c = self.correctness.evaluate_strings(
                    prediction=generated_answer,
                    input=question,
                    reference=reference_answer
                )
                results.update({"correctness_score": c["score"], "correctness_reasoning": c["reasoning"]})
            except Exception as e:
                results.update({"correctness_score": None, "correctness_reasoning": str(e)})

        return results

In [30]:
tools = [retrieve]
agent = create_tool_calling_agent(llm, tools, prompt)

# create the agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


query = "Which are the services offered by the company?"
chatbot_result = agent_executor.invoke({"input": query})
generated_answer = chatbot_result["output"]




> Entering new AgentExecutor chain...

Invoking: `retrieve` with `{'query': 'services offered'}`


ID: 12        
Score: 0.64285713        
Text: In a Full-Service relationship with a dedicated ﬁnancial professional, you have access to a full suite of TIS brokerage and TAS 
investment advisory products and services. Minimum asset balance requirements apply. You have a direct relationship with
your ﬁnancial professional for investment strategies, recommendations, and guidance speciﬁc to your needs and goals. Full- 
Service brokerage accounts are eligible to invest in equities, ﬁxed income, ETFs, mutual funds, annuities, structured products,
alternative investments, and other securities on our Platform which TIS makes available to such accounts. 
In a Full-Service relationship with the Client Advisory Center (“CAC”), you have access to a more limited menu of TIS brokerage
products and services from which a CAC ﬁnancial professional can make recommendations, including mutual funds, annu

In [31]:
evaluator = ContextBasedEvaluator()
results = evaluator.evaluate(
    question=query,
    generated_answer=generated_answer,
    context=serialized_1,
    reference_answer=reference_answer
)
# 4. Output result
print(results)

NameError: name 'reference_answer' is not defined

In [ ]:
import textwrap

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)

# If your results is a dictionary as in your code:
# results = {
#     "question": ...,
#     "generated_answer": ...,
#     "reference_answer": ...,
#     "context": ...,
#     "faithfulness_score": ...,
#     "faithfulness_reasoning": ...,
#     "relevance_score": ...,
#     "relevance_reasoning": ...,
#     "helpfulness_score": ...,
#     "helpfulness_reasoning": ...,
#     "correctness_score": ...,
#     "correctness_reasoning": ...
# }

# Create a DataFrame with explicit columns for each category
df = pd.DataFrame([{
    "Question": results.get("question"),
    "Generated Answer": results.get("generated_answer"),
    "Reference Answer": results.get("reference_answer"),
    "Context": results.get("context"),
    "Faithfulness Score": results.get("faithfulness_score"),
    "Faithfulness Reasoning": results.get("faithfulness_reasoning"),
    "Relevance Score": results.get("relevance_score"),
    "Relevance Reasoning": results.get("relevance_reasoning"),
    "Helpfulness Score": results.get("helpfulness_score"),
    "Helpfulness Reasoning": results.get("helpfulness_reasoning"),
    "Correctness Score": results.get("correctness_score"),
    "Correctness Reasoning": results.get("correctness_reasoning"),
}])
def wrap_text(x):
    if isinstance(x, str):
        return "\n".join(textwrap.wrap(x, width=80))
    return x

df_wrapped = df.applymap(wrap_text)

from IPython.display import display
display(df_wrapped)

In [ ]:

print(response)

In [ ]:
# combine the tools and provide to the llm
tools = [retrieve]
agent = create_tool_calling_agent(llm, tools, prompt)

# create the agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
# invoke the agent
response = agent_executor.invoke({"input": "which are the services offered by the company?"})

In [ ]:
print(response['output'])

In [ ]:
# invoke the agent
response = agent_executor.invoke({"input": "which are the main risk factors highlighted in the document?"})

In [ ]:
print(response['output'])

### Appendix

Qdrant documentation at https://api.qdrant.tech/api-reference/search/query-points

In [ ]:
from qdrant_client import QdrantClient, models

client = QdrantClient(url="http://localhost:6333")

# Query nearest by ID
nearest = client.query_points(
    collection_name="{collection_name}",
    query="43cf51e2-8777-4f52-bc74-c2cbde0c8b04",
)

# Recommend on the average of these vectors
recommended = client.query_points(
    collection_name="{collection_name}",
    query=models.RecommendQuery(recommend=models.RecommendInput(
        positive=["43cf51e2-8777-4f52-bc74-c2cbde0c8b04", [0.11, 0.35, 0.6, ...]],
        negative=[[0.01, 0.45, 0.67, ...]]
    ))
)

# Fusion query
hybrid = client.query_points(
    collection_name="{collection_name}",
    prefetch=[
        models.Prefetch(
            query=models.SparseVector(indices=[1, 42], values=[0.22, 0.8]),
            using="sparse",
            limit=20,
        ),
        models.Prefetch(
            query=[0.01, 0.45, 0.67, ...],  # <-- dense vector
            using="dense",
            limit=20,
        ),
    ],
    query=models.FusionQuery(fusion=models.Fusion.RRF),
)

# 2-stage query
refined = client.query_points(
    collection_name="{collection_name}",
    prefetch=models.Prefetch(
        query=[0.01, 0.45, 0.67, ...],  # <-- dense vector
        limit=100,
    ),
    query=[
        [0.1, 0.2, ...],  # <─┐
        [0.2, 0.1, ...],  # < ├─ multi-vector
        [0.8, 0.9, ...],  # < ┘
    ],
    using="colbert",
    limit=10,
)

# Random sampling (as of 1.11.0)
sampled = client.query_points(
    collection_name="{collection_name}",
    query=models.SampleQuery(sample=models.Sample.RANDOM)
)

# Score boost depending on payload conditions (as of 1.14.0)
tag_boosted = client.query_points(
    collection_name="{collection_name}",
    prefetch=models.Prefetch(
        query=[0.2, 0.8, ...],  # <-- dense vector
        limit=50
    ),
    query=models.FormulaQuery(
        formula=models.SumExpression(sum=[
            "$score",
            models.MultExpression(mult=[0.5, models.FieldCondition(key="tag", match=models.MatchAny(any=["h1", "h2", "h3", "h4"]))]),
            models.MultExpression(mult=[0.25, models.FieldCondition(key="tag", match=models.MatchAny(any=["p", "li"]))])
        ]
    ))
)

# Score boost geographically closer points (as of 1.14.0)
geo_boosted = client.query_points(
    collection_name="{collection_name}",
    prefetch=models.Prefetch(
        query=[0.2, 0.8, ...],  # <-- dense vector
        limit=50
    ),
    query=models.FormulaQuery(
        formula=models.SumExpression(sum=[
            "$score",
            models.GaussDecayExpression(
                gauss_decay=models.DecayParamsExpression(
                    x=models.GeoDistance(
                        geo_distance=models.GeoDistanceParams(
                            origin=models.GeoPoint(
                                lat=52.504043,
                                lon=13.393236
                            ),  # Berlin
                            to="geo.location"
                        )
                    ),
                    scale=5000  # 5km
                )
            )
        ]),
        defaults={"geo.location": models.GeoPoint(lat=48.137154, lon=11.576124)}  # Munich
    )
)
